# HW02
Deep Learning, GIST RT5101-01, 2024, Spring, (Tue/Thurs 2:30~3:45)
***

### Problem1. Fully Connected Layer vs Convolution Neural Network
- Build your custom CNN model
- Check the result CNN has better result than FCN
- The test accuracy must bigger than 60%

----

### Problem2. Train Dogs and Cats data via CNN
- Understand the process of training the CNN model with custom dataloader.   
(Download URL: https://www.kaggle.com/c/dogs-vs-cats)
- Check the result
- The test accuracy must bigger than 60%

***
### You can add additional code for checking your image and model.
### You must summit ``.ipynb`` file. Do not summit ``.py`` file.
---

### How to submit your homework
Submit your jupyter notebook file with the filename of  *HW02_studentnumber.ipynb*  on GEL

Ex) HW02_20222015.ipynb  

### Submission deadline
2024.06.16, Sunday 23:59 (PM)

### Plagiarism
We encourage you to discuss this homework with your friends or TA, but you should write your own code.




***
***

## Problem 1. (total 10 pt.)
- **Fully Connected Layer vs Convolution Neural Network**
- We will use cifar10 dataset.
- You have to compare with HW1 result and check CNN model has better result.
- The test accuracy of CNN model must bigger than 60%.   
- Reference : https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

### Problem 1-1. (2 pt.)
- **Step 1**. Import package.  
- **Step 2**. Define device and configure hyperparameters.  
- **Step 3**. Download then load Cifar10 dataset to dataloader. You have to adjust transform.   

In [18]:
''' Step 1 '''
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

In [40]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

lr = 1e-4
num_classes = 10
batch_size = 32
epochs = 20

In [41]:
def dataset(is_train):

    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5,0.5, 0.5))])

    dataset = torchvision.datasets.CIFAR10(root='./data',train=is_train,download=True,transform=transform)


    return dataset

train_dataset = dataset(is_train=True)
val_dataset = dataset(is_train=False)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, 
                                         shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


### Problem 1-2. (5 pt.)
- **Step 1**. Build your CNN model.  
- **Step 2**. Configure optimizer and objective function.  

In [42]:
''' Step 1 '''
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [43]:
''' Step 2 '''
model = CNN()
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr)
criterion = nn.CrossEntropyLoss().to(device)

### Problem 1-3. (3 pt.)
- **Step 1**. The method for model training
- **Step 2**. The method for testing model
- **Step 3**. Train the model and check the test results
- **Step 4**. Check the output after training

In [44]:
''' Step 1 '''
def train():
    for epoch in range (epochs):
        model.train()
        running_loss=0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs.to(device))
            loss = criterion(outputs, labels.to(device))
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}")

In [45]:
''' Step 2 '''
def test():
    model.eval()
    with torch.no_grad():
        total = 0.0
        correct = 0.0
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Test Accuracy: {(correct/total)*100:.2f}%")

In [46]:
''' Step 3 '''
train()


Epoch 1/20, Loss: 1.6642298261400834
Epoch 2/20, Loss: 1.3475856946892106
Epoch 3/20, Loss: 1.2005108255113612
Epoch 4/20, Loss: 1.0748103195783503
Epoch 5/20, Loss: 0.9850113754537879
Epoch 6/20, Loss: 0.9062769708157501
Epoch 7/20, Loss: 0.8397202224435519
Epoch 8/20, Loss: 0.7804337398256923
Epoch 9/20, Loss: 0.7263063634730881
Epoch 10/20, Loss: 0.6724846528095842
Epoch 11/20, Loss: 0.624682943083427
Epoch 12/20, Loss: 0.5771145732049674
Epoch 13/20, Loss: 0.5297215951324196
Epoch 14/20, Loss: 0.48317391430614703
Epoch 15/20, Loss: 0.4379623132356832
Epoch 16/20, Loss: 0.3968416948398183
Epoch 17/20, Loss: 0.35248496784797023
Epoch 18/20, Loss: 0.31053741542015867
Epoch 19/20, Loss: 0.27234610580551416
Epoch 20/20, Loss: 0.23259805575232437


In [47]:
''' Step 4 '''
test()

Test Accuracy: 74.90%


---
---

## Problem 2. (Total 10 pt.)
- **Train Dogs and Cats data via CNN**
- **You must set the class that Dogs are 0 and Cats are 1.**
- Understand the process of training the CNN model with custom dataloader.
- Download the dataset from below.   
https://www.kaggle.com/c/dogs-vs-cats
- The test accuracy of CNN model must bigger than 60%.   

### Problem 2-1. (4 pt.)
- **Step 1**. Import package.  
- **Step 2**. Define device and configure hyperparameters.  
- **Step 3**. Load **Dogs and Cats** dataset to dataloader. You have to adjust transform.  
**You can label dataset via images name at train folder.**

In [ ]:
''' Step 1 '''

In [ ]:
''' Step 2 '''

In [ ]:
''' Step 3 '''

### Problem 2-2. (3 pt.)
- **Step 1**. Build your CNN model.  
(It doesn't matter if you use same model at problem 1.)
- **Step 2**. Configure optimizer and objective function.

In [ ]:
''' Step 1 '''

In [ ]:
''' Step 2 '''

### Problem 2-3. (3 pt.)
- **Step 1**. The method for model training
- **Step 2**. The method for validation model
- **Step 3**. Train the model and check the validation results
- **Step 4**. Check the test result by **ten** samples with image

In [ ]:
''' Step 1 '''

In [ ]:
''' Step 2 '''

In [ ]:
''' Step 3 '''

In [ ]:
''' Step 4 '''